In [ ]:
import os

import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brute
from scipy.linalg import block_diag
import mhkit.wave.resource as mhkit

import wecopttool as wot
wot.set_loglevel('INFO')

## Given properties

In [ ]:
buoy_props = {
    'CG': 0.242, # m
    'MOI': 7484., # kg-m^2
    'f0': 2 * np.pi /3.0842, # rad/s
}

flywheel_props = {
    'MOI': 25,  # kg-m^2
    'Coulomb friction': 2.0,  # N-m
    'Viscous friction': 0.02,  # N-m/rad
    'Motor damping': 4.0, # N-m-s/rad
}

spring_props = {
    'Max torque': 750, # N-m
    'Max displacement': np.deg2rad(45.0), # rad
}

## Geometry
Same general shape as WaveBot

In [ ]:
in2m = 0.0254 # inch -> meter conversion factor
draft = 0.5 # meters
freeboard = 40. * in2m - draft
hull_geom = wot.geom.WaveBot(r1=130./2 * in2m,
                             r2=110./2 * in2m, 
                             h1=22.679 * in2m,
                             h2=17.321 * in2m,
                             scale_factor=1,
                             freeboard=freeboard)
mesh = hull_geom.mesh(mesh_size_factor=0.5)
_ = hull_geom.plot_cross_section()

### Floating body
1DOF in pitch. The `FloatingBody` object in Capytaine only models the buoy since it is the only body being excited by the waves; the flywheel dynamics will be captured later on.

In [ ]:
pnr_fb = cpy.FloatingBody.from_meshio(mesh, name="Pioneer")
pnr_fb.add_rotation_dof(name='Pitch')
pnr_fb.center_of_mass = np.array([0., 0., buoy_props['CG']])
pnr_fb.rotation_center = pnr_fb.center_of_mass
ndof = pnr_fb.nb_dofs
pnr_fb.show_matplotlib()

## Hydrodynamics and hydrostatics
Hydrostatic stiffness values match closely to experimental values

In [ ]:
rho = 1025. # kg/m^3
f1 = 0.025 # Hz
nfreq = 25
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency


file_name = "data/pioneer_bem_25freq.nc"
if os.path.exists(file_name):
    bem_data = wot.read_netcdf(file_name)
else:    
    bem_data = wot.run_bem(pnr_fb, freq)
    wot.write_netcdf(file_name, bem_data)
omega = bem_data.omega.values

pnr_fb.keep_immersed_part()
k_buoy = pnr_fb.compute_hydrostatic_stiffness(rho=rho).values.squeeze()
k_spring = spring_props['Max torque'] / spring_props['Max displacement']
print(f'Hydrostatic stiffness from Capytaine: {k_buoy} N-m/rad')
print('Hydrostatic stiffness from experiment: 37204 N-m/rad')

In [ ]:
# plots
fig_am, ax_am = plt.subplots(tight_layout=True, sharex=True)
fig_rd, ax_rd = plt.subplots(tight_layout=True, sharex=True)
fig_ex, ax_ex = plt.subplots(tight_layout=True, sharex=True)

# Excitation
np.abs(bem_data.diffraction_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashed', label='Diffraction force')
np.abs(bem_data.Froude_Krylov_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashdot', label='Froude-Krylov force')
ex_handles, ex_labels = ax_ex.get_legend_handles_labels()
ax_ex.set_xlabel(f'$\omega$', fontsize=10)
ax_ex.set_title('Wave Excitation Coefficients', fontweight='bold')
fig_ex.legend(ex_handles, ex_labels, loc='center right', frameon=False)

# Added mass
bem_data.added_mass.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_am)
ax_am.set_xlabel(f'$\omega$', fontsize=10)
ax_am.set_title('Added Mass Coefficients', fontweight='bold')

# Radiation damping
bem_data.radiation_damping.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_rd)
ax_rd.set_xlabel(f'$\omega$', fontsize=10)
ax_rd.set_title('Radiation Damping Coefficients', fontweight='bold')

## PTO
Using same impedance matrix as WaveBot, except replace gear ratio with the one determined from the spring stiffness and buoy moment of inertia + natural frequency.

In [ ]:
nstate_pto = 2 * nfreq

def gear_ratio(k_spring, moi, nat_freq):
    return np.sqrt(k_spring / (moi * nat_freq**2))
n_default = gear_ratio(k_spring, flywheel_props['MOI'], buoy_props['f0'])

In [ ]:
torque_constant = 3.512
winding_resistance = 0.304
winding_inductance = 0.0
drivetrain_inertia = 2.0 #flywheel_props['MOI']
drivetrain_friction = 1.0
drivetrain_stiffness = 0.0

drivetrain_impedance = (1j*omega*drivetrain_inertia + 
                        drivetrain_friction + 
                        1/(1j*omega)*drivetrain_stiffness) 

winding_impedance = winding_resistance + 1j*omega*winding_inductance


z_11 = np.array([[-1* n_default**2 * drivetrain_impedance]])
off_diag = np.sqrt(3.0/2.0) * torque_constant * n_default
z_12 = np.array([[-1*(off_diag+0j) * np.ones(omega.shape)]])
z_21 = np.array([[-1*(off_diag+0j) * np.ones(omega.shape)]])
z_22 = np.array([[winding_impedance]])
z_12_inv = np.linalg.inv(z_12.T).T

mmult = lambda a,b: np.einsum('mnr,mnr->mnr', a, b)
abcd_11 = -1 * mmult(z_12_inv, z_11)
abcd_12 = z_12_inv
abcd_21 = z_21 - mmult(z_22, mmult(z_12_inv, z_11))
abcd_22 = mmult(z_22, z_12_inv)

row_1 = np.hstack([abcd_11, abcd_12])
row_2 = np.hstack([abcd_21, abcd_22])
abcd = np.vstack([row_1, row_2])

In [ ]:
def block(re, im): return np.array([[re, -im], [im, re]])
for idof in range(2):
    for jdof in range(2):
        Zp = abcd[idof, jdof, :]
        re = np.real(Zp)
        im = np.imag(Zp)
        # Exclude the sine component of the 2-point wave
        blocks = [block(ire, iim) for (ire, iim) in zip(re[:-1], im[:-1])]
        # re[0] added for the zero frequency power loss (DC), could be re[n]
        blocks = [re[0]] + blocks + [re[-1]]
        if jdof==0:
            row = block_diag(*blocks)
        else:
            row = np.hstack([row, block_diag(*blocks)])
    if idof==0:
        transfer_mat = row
    else:
        transfer_mat = np.vstack([transfer_mat, row])
transfer_mat = None

In [ ]:
def x_rel(wec, x_wec, x_opt):
    pos_wec = wec.vec_to_dofmat(x_wec)
    pos_fw = wec.vec_to_dofmat(x_opt[nstate_pto:])
    return pos_wec - pos_fw

def rel_position(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_rel = x_rel(wec, x_wec, x_opt)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, pos_rel)

def rel_velocity(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_rel = x_rel(wec, x_wec, x_opt)
    vel_rel = np.dot(wec.derivative_mat, pos_rel)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, vel_rel)

def f_motor(wec, x_wec, x_opt, waves, nsubsteps=1):
    motor = x_opt[:nstate_pto]
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, motor)

def mechanical_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    force_td = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    vel_td = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return vel_td * force_td

def mechanical_energy(wec, x_wec, x_opt, waves, nsubsteps=1):
    power_td = mechanical_power(wec, x_wec, x_opt, waves, nsubsteps)

def mechanical_average_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    energy = mechanical_energy(wec, x_wec, x_opt, waves, nsubsteps)
    return energy / wec.tf

def power(wec, x_wec, x_opt, waves, nsubsteps=1):
    if transfer_mat is not None:
        q1_td = rel_velocity(wec, x_wec, x_opt, waves)
        e1_td = f_motor(wec, x_wec, x_opt, waves)
        q1 = wec.complex_to_real(wec.td_to_fd(q1_td, False))
        e1 = wec.complex_to_real(wec.td_to_fd(e1_td, False))
        vars_1 = np.hstack([q1, e1])
        vars_1_flat = wec.dofmat_to_vec(vars_1)
        vars_2_flat = np.dot(transfer_mat, vars_1_flat)
        vars_2 = wec.vec_to_dofmat(vars_2_flat, 2)
        q2 = vars_2[:, 0]
        e2 = vars_2[:, 1]
        time_mat = wec.time_mat_substeps(nsubsteps)
        q2_td = np.dot(time_mat, q2)
        e2_td = np.dot(time_mat, e2)
    else:
        q2_td = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
        e2_td = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    return q2_td * e2_td

def energy(wec, x_wec, x_opt, waves, nsubsteps=1):
    power_td = power(wec, x_wec, x_opt, waves, nsubsteps)
    return np.sum(power_td) * wec.dt/nsubsteps

def average_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    energy = energy(wec, x_wec, x_opt, waves, nsubsteps)
    return energy / wec.tf

In previous tutorials, all the relevant WEC dynamics have been captured in the `x_wec` vector. However, `x_wec` assumes all the degrees of freedom are excited by waves, which is not the case with the flywheel in the Pioneer model. Since we need to capture the flywheel dynamics to properly model the PTO, we work around this by including the position states of the flywheel as appended entries in `x_opt`, and overwrite the motion functions within the `PTO` module to capture the relative motion of the flywheel so the power calculations within that module are correct. This is a dirty hack for now -- this will be more cleanly implemented in an upcoming release, and this tutorial will be updated accordingly.

### Constraints
 * **Peak torque** - Peak torque
 * **Peak RMS speed** - Peak RMS speed

In [ ]:
torque_peak_max = 50.
permitted_rms_speed = 3300 * 2*np.pi / 60 # rad/s
def const_peak_torque_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    torque = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    return torque_peak_max - np.abs(torque.flatten())

def const_rms_velocity_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    pos = wec.vec_to_dofmat(x_opt[nstate_pto:])
    pto_vel = np.dot(wec.derivative_mat, pos)
    rms_vel = np.sqrt(np.mean(pto_vel**2))
    return permitted_rms_speed - rms_vel

def const_peak_spring_angle(wec, x_wec, x_opt, waves, nsubsteps): # temporary, for validation
    rel_pos = rel_position(wec, x_wec, x_opt, waves, nsubsteps)
    return spring_props['Max displacement'] - np.abs(rel_pos)

# min_line_tension = 100
# def const_min_tension(wec, x_wec, x_opt, waves, nsubsteps=2):
#     total_tension = -1*f_pto_line(wec, x_wec, x_opt, waves, nsubsteps)
#     return total_tension.flatten() - min_line_tension #TODO

### Additional forces
 * **Flywheel friction** - Flywheel friction
 * **Magnetic spring** - Magnetic spring
 * **Flywheel inertia** - Not really a force, but captured here for the flywheel residual function below

In [ ]:
def f_friction(wec, x_wec, x_opt, waves, nsubsteps=1):
    rel_vel = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
    fric = 1 * (
        np.tanh(rel_vel) * flywheel_props['Coulomb friction']
      + flywheel_props['Viscous friction'] * rel_vel)
    return fric

def f_spring(wec, x_wec, x_opt, waves, nsubsteps=1):
    rel_pos = rel_position(wec, x_wec, x_opt, waves, nsubsteps)
    return -k_spring * (1/n_default)**2 * rel_pos

def fw_inertia(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_fw = wec.vec_to_dofmat(x_opt[nstate_pto:])
    acc_fw = np.dot(wec.derivative2_mat, pos_fw)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    acc_fw = np.dot(time_matrix, acc_fw)
    return flywheel_props['MOI'] * acc_fw

#### Residual
Equality constraint of Newton's 2nd law on the flywheel (i.e. $r(x) = I \alpha - \tau = 0 $). Same basic structure as residual for the WEC dynamics as described in the [WecOptTool theory documentation](https://sandialabs.github.io/WecOptTool/theory.html).

In [ ]:
def const_flywheel_resid(wec, x_wec, x_opt, waves):
    inertia = fw_inertia(wec, x_wec, x_opt, waves)
    fs = f_spring(wec, x_wec, x_opt, waves)
    ff = f_friction(wec, x_wec, x_opt, waves)
    fp = f_motor(wec, x_wec, x_opt, waves)
    return (inertia +fs -ff +fp).flatten()

### WEC object
Finally, we can use all the different components we've developed thus far to construct a `WEC` object:

In [ ]:
f_add = {
    'Spring': f_spring,
    'PTO': f_motor,
}

constraints = [
    # {'type': 'ineq', 'fun': const_peak_torque_pto},
    # {'type': 'ineq', 'fun': const_rms_velocity_pto},
    {'type': 'eq', 'fun': const_flywheel_resid},
    # {'type': 'ineq', 'fun': const_min_tension},
]

In [ ]:
wec = wot.WEC.from_bem(
    bem_data,
    inertia_matrix=np.array([[buoy_props['MOI']]]),
    hydrostatic_stiffness=k_buoy,
    f_add=f_add,
    constraints=constraints,
    dof_names=bem_data.influenced_dof.values,
)

## Waves
A regular wave will allow us to get a good initial understanding of the optimal control trajectory.
Note that we'll want to choose a wave frequency that is within the frequency array we used to calculate the hydrodynamic data.

In [ ]:
amplitude = 0.15
wavefreq =  0.3
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude)

gamma = 1.25
te = 5.6
fp = 1 / mhkit.energy_period_to_peak_period(te, gamma)
hs = 1.25
spectrum = lambda f: wot.waves.jonswap_spectrum(freq=f, fp=fp, hs=hs, gamma=gamma)
efth = wot.waves.omnidirectional_spectrum(f1, nfreq, spectrum, "JONSWAP")
# waves = wot.waves.long_crested_wave(efth)
efth.plot(marker='.')

## Solve
Note the size of the `nstate_opt` variable compared to previous tutorials. The first half of the variable includes the PTO force states at each Fourier component, as in previous tutorials (the `nstate_pto` component). The second half is the flywheel positions at each Fourier component, which is basically an analog to `wec.nstate_wec`.

In [ ]:
obj_fun = average_power
nstate_opt = nstate_pto + wec.nstate_wec
results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    scale_x_wec=1,
    scale_x_opt=1e-3,
    scale_obj=1e-1,
)

## Post-process and plotting

In [ ]:
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps=5)
print(f'Optimal average power: {results.fun:.2f} W')

In [ ]:
x_wec, x_opt = wot.decompose_state(results.x, 1, nfreq)
pto_pos = rel_position(wec, x_wec, x_opt, waves, nsubsteps=5)
pto_vel = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps=5)
pto_force = f_motor(wec, x_wec, x_opt, waves, nsubsteps=5)
pto_mech_power = mechanical_power(wec, x_wec, x_opt, waves, nsubsteps=5)
fw_pos = np.dot(wec.time_mat_nsubsteps(5), x_opt[nstate_pto:])

In [ ]:
Fex = np.abs((wec_fdom.force.sel(type='Froude_Krylov') + wec_fdom.force.sel(type='diffraction')).squeeze())
plt.plot(wec_fdom.omega, Fex)

In [ ]:
wec_tdom.force.sel(influenced_dof='Pitch').plot(hue='type')

In [ ]:
pto_tdom.force.plot()

In [ ]:
fig, ax = plt.subplots(nrows=4, sharex=True, figsize=(12, 12))

ax[0].set_ylabel('Exc. torque [Nm]')
pto_tdom.force.plot(ax=ax[0], label='PTO', color='r')
wec_tdom.force.sel(influenced_dof='Pitch').sum(dim='type').plot(ax=ax[0], label='Buoy', color='c')
ax[0].set_ylabel('Torque [Nm]', color='red')
ax[0].tick_params(axis='y', labelcolor='red')
ax[0].set_title('')
ax[0].autoscale(enable=True, axis='x', tight=False)
ax[0].legend()

wec_pos = np.rad2deg(wec_tdom.pos)
spring_pos = np.rad2deg(pto_tdom.pos)
fw_pos = spring_pos + wec_pos
wec_pos.plot(ax=ax[1], hue='influenced_dof', label='Buoy')
fw_pos.plot(ax=ax[1], hue='influenced_dof', label='Flywheel')
spring_pos.plot(ax=ax[1], linestyle='dashed', label='Spring angle')
ax[1].set_ylabel('Position [deg]')
ax[1].legend()

wec_tdom.vel.plot(ax=ax[2], hue='influenced_dof', label='Buoy')
pto_tdom.vel.plot(ax=ax[2], hue='influenced_dof', label='PTO')
(pto_tdom.vel + wec_tdom.vel).plot(ax=ax[2], hue='influenced_dof', label='Flywheel')
ax[2].legend()

(pto_tdom['mech_power']).plot(ax=ax[3], 
                                  label='Mech. power ($\\bar{P}_{mech}$: ' + f'{pto_tdom.mech_power.mean().item():.0f} W)')
(pto_tdom['power']).plot(ax=ax[3], linestyle='dashed', 
                         label='Elec. power ($\\bar{P}_{elec}$: ' + f'{pto_tdom.power.mean().item():.0f} W)')

ax[3].legend(loc='upper right')
ax[3].set_ylabel('Power [W]')

for axi in ax:
    axi.set_title('')
    axi.grid()
    axi.label_outer()
    axi.autoscale(axis='x', tight=True)

# fig.savefig('../gfx/PioneerWec_wecopttool_time_histories.pdf')

In [ ]:
np.abs(pto_fdom.force).plot()